## Robust plan

In [1]:
using JuMP, Gurobi, Plots, LinearAlgebra
GRB_ENV = Gurobi.Env();

Academic license - for non-commercial use only


### Create model for dynamics

In [268]:
dt = 0.1
A = zeros(4,4)+I
A[1,2] = A[3,4] = dt
B = zeros(4,2)
B[2,1] = B[4,2] = dt
C = zeros(2,4)
C[1,2] = C[2,4] = 1
D = zeros(2,2)
Bw = copy(B);

### Create model

In [263]:
T = 0.9
N = Int(T/dt);
x0 = [0;0;0;0]
xN = [0.03;0;0.035;0]
P = [-1 0; 0 1]
q = [-0.015; 0.03];
m = 2
N1 = 2
M = 1000;
ϵ = 0.00001

1.0e-5

In [265]:
Q = zeros(2,4)
Q[1,2] = Q[2,4] = 1;

In [146]:
N_K = Int((N-1)*N/2)

36

In [216]:
Γ = 2
μ = [0;0]
σ = [4;1]*1e-3
Ak = zeros(N,4,4)
Ak[1,:,:] = zeros(Int, size(A))+I
for i=2:N
    Ak[i,:,:] = A*Ak[i-1,:,:]
end
η = k -> A1*C*Ak[k+1,:,:]
cki = (k,i) -> A1*C*(Ak[k-i,:,:]*Bw)
dki = (k,i) -> A1*C*(Ak[k-i,:,:]*B); #defined for k=>2
γ_1 = Γ*σ*sqrt(N)+N*μ
γ_2 = Γ*σ*sqrt(N)-N*μ
β = Γ*σ;
ρ = 3;

In [ ]:
ABwki = (k,i) -> Ak[k-i,:,:]*Bw
ABki = (k,i) -> Ak[k-i,:,:]*B

In [150]:
#up_low = k-> Int((k-1)*(k-2)/2+1):Int(k*(k-1)/2);

In [151]:
getK = (i,j) -> Int(i*(i-1)/2)+j;

## Probabilistic formulation

In [152]:
k,i,j = (2,0,1)
dki(k,j)*K[getK(j,i+1),:,:]

2×2 Array{GenericAffExpr{Float64,VariableRef},2}:
 -0.1 K[1,1,1]  -0.1 K[1,1,2]
 0.1 K[1,2,1]   0.1 K[1,2,2]

In [259]:
model = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GRB_ENV), #"OutputFlag" => 0
    ));
@variable(model, K[1:N_K,1:2,1:2])
@variable(model, α[1:2, 1:N])
@variable(model, au[1:2, 1:N-1] >=0)
@variable(model, t[1:N_K,1:m] >= 0)
@variable(model, s[1:N_K,1:m] >= 0)
@variable(model, z[1:N,1:2] >=0, Bin)
@constraint(model, sum(z, dims=2) .== N1-1);

In [261]:
sum(cki(k,i)*α[:,i+1] for i=0:k-1)

2-element Array{GenericAffExpr{Float64,VariableRef},1}:
 -0.1 α[1,1] - 0.1 α[1,2]
 0.1 α[2,1] + 0.1 α[2,2]

In [262]:
sum(sum(dki(k,j)*K[getK(j,i+1),:,:] for j=(i+1):(k-1)) for i=0:k-2)

2×2 Array{GenericAffExpr{Float64,VariableRef},2}:
 -0.1 K[1,1,1]  -0.1 K[1,1,2]
 0.1 K[1,2,1]   0.1 K[1,2,2]

In [234]:
@constraint(model, [k=2:N-1], sum(cki(k,i)*α[:,i+1] for i=0:k-1)+ρ*sum(s[i] for i=0:k-2) .<= au)
@constraint(model, [k=2:N-1], sum(cki(k,i)*α[:,i+1] for i=0:k-1)+ρ*sum(s[i] for i=0:k-2) .>= -au)
@constraint(model, [k=2:N,i=0:(k-2)], vcat(s[k-2+i+1,l], sum(
            dki(k,j)*K[getK(j,i+1),:,:][l,:] for j=(i+1):(k-1))) in SecondOrderCone());
@constraint(model, [k=2:N,i=0:(k-2),l=1:2], vcat(t[k-2+i+1,l], cki(k,i)[l,:]+sum(
            dki(k,j)*K[getK(j,i+1),:,:][l,:] for j=(i+1):(k-1))) in SecondOrderCone());
@constraint(model, [k=2:N-1], (η(k)*x0+sum(cki(k,i)*α[:,i+1] for i=0:k-1)
        +sum(ρ*t[k-2+i+1,:] for i=0:k-2)
        +ρ*mapslices(norm, A1*C*Bw, dims=2)[:]) .<= b1+M*z[k,:]);
@objective(model, Min, sum(au));

In [184]:
j = 1
(dki(k,j)*K[getK(j,i+1),:,:])[1,:]

2-element Array{GenericAffExpr{Float64,VariableRef},1}:
 -0.1 K[1,1,1]
 -0.1 K[1,1,2]

In [185]:
cki(k,i)[1,:]+(dki(k,j)*K[getK(j,i+1),:,:])[1,:]

2-element Array{GenericAffExpr{Float64,VariableRef},1}:
 -0.1 K[1,1,1] - 0.1
 -0.1 K[1,1,2]

In [173]:
K[getK(j,i+1),:,1]

2-element Array{VariableRef,1}:
 K[1,1,1]
 K[1,2,1]

In [175]:
sum(dki(k,j)[1,:]*K[getK(j,i+1),:,:] for j=(i+1):(k-1))

-0.1 K[1,1,1]

In [192]:
k,i = (2,0)
vcat(t[k,1], cki(k,i)[1,:]+sum(
        dki(k,j)*K[getK(j,i+1),:,:][1,:] for j=(i+1):(k-1)))

3-element Array{GenericAffExpr{Float64,VariableRef},1}:
 t[2,1]
 -0.1 K[1,1,1] - 0.1
 0.1 K[1,1,2]

In [223]:
@constraint(model, [k=2:N,i=0:(k-2),l=1:2], vcat(t[k-2+i+1,l], cki(k,i)[l,:]+sum(
            dki(k,j)*K[getK(j,i+1),:,:][l,:] for j=(i+1):(k-1))) in SecondOrderCone());

In [200]:
cki(k,i)

2×2 Array{Float64,2}:
 -0.1  0.0
  0.0  0.1

In [243]:
η(k)*x0

2-element Array{Float64,1}:
 0.0
 0.0

In [246]:
η(k)*x0+sum(cki(k,i)*α[:,i+1] for i=0:k-1)+sum(ρ*t[k-2+i+1,:] for i=0:k-2)

2-element Array{GenericAffExpr{Float64,VariableRef},1}:
 -0.1 α[1,1] - 0.1 α[1,2] + 3 t[1,1]
 0.1 α[2,1] + 0.1 α[2,2] + 3 t[1,2]

In [224]:
sum(ρ*t[k-2+i+1,:] for i=0:k-2)

2-element Array{GenericAffExpr{Float64,VariableRef},1}:
 3 t[1,1]
 3 t[1,2]

In [255]:
(η(k)*x0+sum(cki(k,i)*α[:,i+1] for i=0:k-1)
        +sum(ρ*t[k-2+i+1,:] for i=0:k-2)
        +ρ*mapslices(norm, A1*C*Bw, dims=2)[:])

2-element Array{GenericAffExpr{Float64,VariableRef},1}:
 -0.1 α[1,1] - 0.1 α[1,2] + 3 t[1,1] + 0.30000000000000004
 0.1 α[2,1] + 0.1 α[2,2] + 3 t[1,2] + 0.30000000000000004

In [239]:
b1+M*z[k,:]

2-element Array{GenericAffExpr{Float64,VariableRef},1}:
 1000 z[2,1] - 0.015
 1000 z[2,2] + 0.03

In [257]:
@constraint(model, [k=2:N-1], (η(k)*x0+sum(cki(k,i)*α[:,i+1] for i=0:k-1)
        +sum(ρ*t[k-2+i+1,:] for i=0:k-2)
        +ρ*mapslices(norm, A1*C*Bw, dims=2)[:]) .<= b1+M*z[k,:]);

1-dimensional DenseAxisArray{Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1},1,...} with index sets:
    Dimension 1, 2:8
And data, a 7-element Array{Array{ConstraintRef{Model,MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64},MathOptInterface.LessThan{Float64}},ScalarShape},1},1}:
 [-0.1 α[1,1] - 0.1 α[1,2] + 3 t[1,1] - 1000 z[2,1] <= -0.31500000000000006, 0.1 α[2,1] + 0.1 α[2,2] + 3 t[1,2] - 1000 z[2,2] <= -0.27]
 [-0.1 α[1,1] - 0.1 α[1,2] - 0.1 α[1,3] + 3 t[2,1] + 3 t[3,1] - 1000 z[3,1] <= -0.31500000000000006, 0.1 α[2,1] + 0.1 α[2,2] + 0.1 α[2,3] + 3 t[2,2] + 3 t[3,2] - 1000 z[3,2] <= -0.27]
 [-0.1 α[1,1] - 0.1 α[1,2] - 0.1 α[1,3] - 0.1 α[1,4] + 3 t[3,1] + 3 t[4,1] + 3 t[5,1] - 1000 z[4,1] <= -0.31500000000000006, 0.1 α[2,1] + 0.1 α[2,2] + 0.1 α[2,3] + 0.1 α[2,4] + 3 t[3,2] + 3 t[4,2] + 3 t[5,2] - 1000 z[4,2] <= -0.27]
 [-0.1 α[1,1] - 0.1 α[1,2]

### Dual formulation

In [4]:
model = Model(optimizer_with_attributes(
        () -> Gurobi.Optimizer(GRB_ENV), #"OutputFlag" => 0
    ));
@variable(model, r1[1:2]>=0)
@variable(model, r2[1:2]>=0)
@variable(model, q[1:2, 1:N]>=0)
@variable(model, p[1:2, 1:N]>=0)
@variable(model, u[1:2, 1:N-1])
@variable(model, au[1:2, 1:N-1] >=0)
@variable(model, z[1:2, 1:N-1] >=0, Bin)
@constraint(model, (dot(γ_1,r1)+dot(γ_2,r2)+dot(β,sum(q+p,dims=2)+
        sum(-M*z+dki*u)))
#@variable(model, g[1:N], Bin)
#@variable(model, ag[1:4, 1:N] >= 0)
@constraint(model, [k=1:N], A1*x[[1,3],k] -b1 + M*z[k,:] .>= 0 )
@constraint(model, sum(z, dims=2) .== N1-1)
@constraint(model, x[:,1] .== x0)
@constraint(model, x[:,N] .== xN)
@constraint(model, [k=1:N-1], x[:,k+1] .== A*x[:,k]+B*u[:,k])
#@constraint(model, [k=1:N-1], y[:,k+1] .== C*x[:,k]) #sensor output
@constraint(model, -au .<= u)
@constraint(model, u .<= au)
@constraint(model, au .<= 0.5)
#@constraint(model, [k=1:N], -ag[:,k] .<= x[:, k] - xN)
#@constraint(model, [k=1:N], x[:, k] - xN .<= ag[:,k])
#@constraint(model, [k=1:N,i=1:4], -1000*g[k]+ag[i,k] .<= epsilon)
#@constraint(model, -1 .<= x[[2,4],:] .<= 1) # velocity constraint
#@objective(model, Min, sum(g));
@objective(model, Min, sum(au));